---
title: Python Type hint
tags: 小书匠,python,built-in,type-hint,mypy,typing
grammar_cjkRuby: true
---

[toc!]

# Python Type hint

So, Type Annotations or Type Comments?

所以向自己的代码添加类型提示时，应该使用注释还是类型注释？简而言之:尽可能使用注释，必要时使用类型注释。
注释提供了更清晰的语法，使类型信息更接近您的代码。它们也是官方推荐的写入类型提示的方式，并将在未来进一步开发和适当维护。

类型注释更详细，可能与代码中的其他类型注释冲突，如linter指令。但是，它们可以用在不支持注释的代码库中。

还有一个隐藏选项3:存根文件。稍后，当我们讨论向第三方库添加类型时，您将了解这些。

存根文件可以在任何版本的Python中使用，代价是必须维护第二组文件。通常，如果无法更改原始源代码，则只需使用存根文件。

In [1]:
from typing import List, Sequence

def square(elems: Sequence[float]) -> List[float]:
    return [x**2 for x in elems]

使用 Sequence 是一个典型的鸭子类型的例子. 也就意味着可以使用len() 和 . __getitem__()等方法。

## Variable

In [2]:
# This is how you declare the type of a variable type in Python 3.6
age: int = 1

# In Python 3.5 and earlier you can use a type comment instead
# (equivalent to the previous definition)
age = 1  # type: int

# You don't need to initialize a variable to annotate it
a: int  # Ok (no value at runtime until assigned)

# The latter is useful in conditional branches
child: bool
if age < 18:
    child = True
else:
    child = False

## Built-in types

In [3]:
from typing import List, Set, Dict, Tuple, Optional

# For simple built-in types, just use the name of the type
x: int = 1
x: float = 1.0
x: bool = True
x: str = "test"
x: bytes = b"test"

# For collections, the type of the collection item is in brackets
# (Python 3.9+)
x: list[int] = [1]
x: set[int] = {6, 7}

# In Python 3.8 and earlier, the name of the collection type is
# capitalized, and the type is imported from 'typing'
x: List[int] = [1]
x: Set[int] = {6, 7}

# Same as above, but with type comment syntax (Python 3.5 and earlier)
x = [1]  # type: List[int]

TypeError: 'type' object is not subscriptable

### dict

In [4]:
# For mappings, we need the types of both keys and values
x: dict[str, float] = {'field': 2.0}  # Python 3.9+
x: Dict[str, float] = {'field': 2.0}

TypeError: 'type' object is not subscriptable

### tuple

In [ ]:
# For tuples of fixed size, we specify the types of all the elements
x: tuple[int, str, float] = (3, "yes", 7.5)  # Python 3.9+
x: Tuple[int, str, float] = (3, "yes", 7.5)

# For tuples of variable size, we use one type and ellipsis
x: tuple[int, ...] = (1, 2, 3)  # Python 3.9+
x: Tuple[int, ...] = (1, 2, 3)

### Optional

`Optional[str]` 相当于 `Union[str, None]`

In [ ]:
# Use Optional[] for values that could be None
x: Optional[str] = '123'
x: Optional[str] = None

### Any

In [ ]:
import random
from typing import Any, Sequence

def choose(items: Sequence[Any]) -> Any:
  return random.choice(items)

names = ["Guido", "Jukka", "Ivan"]
reveal_type(names)

name = choose(names)
reveal_type(name)

虽然Mypy会正确推断名称是字符串列表，但由于使用了任意类型，在调用choose ( )后，该信息会丢失:

```
$ mypy choose.py
choose.py:10: error: Revealed type is 'builtins.list[builtins.str*]'
choose.py:13: error: Revealed type is 'Any'
```

由此可以得知，如果使用了Any使用mypy的时候将不容易检测。

## Functions
Python 3 supports an annotation syntax for function declarations.

In [ ]:
from typing import Callable, Iterator, Union, Optional, List

# This is how you annotate a function definition
def stringify(num: int) -> str:
    return str(num)

# And here's how you specify multiple arguments
def plus(num1: int, num2: int) -> int:
    return num1 + num2

# Add default value for an argument after the type annotation
def f(num1: int, my_float: float = 3.5) -> float:
    return num1 + my_float

# This is how you annotate a callable (function) value
x: Callable[[int, float], float] = f
    
# 对于没有返回值的函数，我们可以指定None
def play(player_name: str) -> None:
    print(f"{player_name} plays")

### 注解 *args 和 **kwargs

在面向对象的游戏版本中，我们添加了在命令行上命名玩家的选项。这是通过在程序名称后面列出玩家名称来完成的：

关于类型注释:即使名称是字符串元组，也应该只注释每个名称的类型。换句话说，您应该使用字符串而不是元组[字符串]，就像下面这个例子:

In [ ]:
class Game:
     def __init__(self, *names: str) -> None:
         """Set up the deck and deal cards to 4 players"""
         deck = Deck.create(shuffle=True)
         self.names = (list(names) + "P1 P2 P3 P4".split())[:4]
         self.hands = {
             n: Player(n, h) for n, h in zip(self.names, deck.deal(4))
         }

类似地，如果有一个接受**kwargs的函数或方法，那么你应该只注释每个可能的关键字参数的类型。

### Generator

In [ ]:
# A generator function that yields ints is secretly just a function that
# returns an iterator of ints, so that's how we annotate it
def g(n: int) -> Iterator[int]:
    i = 0
    while i < n:
        yield i
        i += 1

# You can of course split a function annotation over multiple lines
def send_email(address: Union[str, List[str]],
               sender: str,
               cc: Optional[List[str]],
               bcc: Optional[List[str]],
               subject='',
               body: Optional[List[str]] = None
               ) -> bool:
    ...

In [ ]:
# An argument can be declared positional-only by giving it a name
# starting with two underscores:
def quux(__x: int) -> None:
    pass

quux(3)  # Fine
quux(__x=3)  # Error

## When you’re puzzled or when things are complicated

In [ ]:
from typing import Union, Any, List, Optional, cast

# To find out what type mypy infers for an expression anywhere in
# your program, wrap it in reveal_type().  Mypy will print an error
# message with the type; remove it again before running the code.
reveal_type(1)  # -> Revealed type is 'builtins.int'

# Use Union when something could be one of a few types
x: List[Union[int, str]] = [3, 5, "test", "fun"]

# Use Any if you don't know the type of something or it's too
# dynamic to write a type for
x: Any = mystery_function()

# If you initialize a variable with an empty container or "None"
# you may have to help mypy a bit by providing a type annotation
x: List[str] = []
x: Optional[str] = None

# This makes each positional arg and each keyword arg a "str"
def call(self, *args: str, **kwargs: str) -> str:
    request = make_request(*args, **kwargs)
    return self.do_api_query(request)

# Use a "type: ignore" comment to suppress errors on a given line,
# when your code confuses mypy or runs into an outright bug in mypy.
# Good practice is to comment every "ignore" with a bug link
# (in mypy, typeshed, or your own code) or an explanation of the issue.
x = confusing_function()  # type: ignore  # https://github.com/python/mypy/issues/1167

# "cast" is a helper function that lets you override the inferred
# type of an expression. It's only for mypy -- there's no runtime check.
a = [4]
b = cast(List[int], a)  # Passes fine
c = cast(List[str], a)  # Passes fine (no runtime check)
reveal_type(c)  # -> Revealed type is 'builtins.list[builtins.str]'
print(c)  # -> [4]; the object is not cast

# If you want dynamic attributes on your class, have it override "__setattr__"
# or "__getattr__" in a stub or in your source code.
#
# "__setattr__" allows for dynamic assignment to names
# "__getattr__" allows for dynamic access to names
class A:
    # This will allow assignment to any A.x, if x is the same type as "value"
    # (use "value: Any" to allow arbitrary types)
    def __setattr__(self, name: str, value: int) -> None: ...

    # This will allow access to any A.x, if x is compatible with the return type
    def __getattr__(self, name: str) -> int: ...

a.foo = 42  # Works
a.bar = 'Ex-parrot'  # Fails type checking

## Standard “duck types”

In typical Python code, many functions that can take a list or a dict as an argument only need their argument to be somehow “list-like” or “dict-like”. A specific meaning of “list-like” or “dict-like” (or something-else-like) is called a “duck type”, and several duck types that are common in idiomatic Python are standardized.

- Mapping 支持 `__getitem__` 
- Mapping 支持 `__getitem__` 和 `__setitem__`
- Sequence 支持 `__len__` 和 `__getitem__`
- Iterable anything useable in "for"

Tuple 是不可变序列，通常由固定数量的可能不同类型的元素组成。例如，我们将卡片表示为套装和等级的元组。通常，您为n元组编写Tuple[t_1，t_2，...，t_n]。

List 是可变序列，通常由未知数量的相同类型的元素组成，例如卡片列表。无论列表中有多少元素，注释中只有一种类型:List[t]。

在许多情况下，你的函数会期望某种序列，并不关心它是 List 还是 Tuple。在这些情况下，您应该使用 typing.Sequence 

In [ ]:
from typing import Mapping, MutableMapping, Sequence, Iterable, List, Set

# Use Iterable for generic iterables (anything usable in "for"),
# and Sequence where a sequence (supporting "len" and "__getitem__") is
# required
def f(ints: Iterable[int]) -> List[str]:
    return [str(x) for x in ints]

f(range(1, 3))

In [ ]:
# Mapping describes a dict-like object (with "__getitem__") that we won't
# mutate, and MutableMapping one (with "__setitem__") that we might
def f(my_mapping: Mapping[int, str]) -> List[int]:
    my_mapping[5] = 'maybe'  # if we try this, mypy will throw an error...
    return list(my_mapping.keys())

f({3: 'yes', 4: 'no'})

def f(my_mapping: MutableMapping[int, str]) -> Set[str]:
    my_mapping[5] = 'maybe'  # ...but mypy is OK with this.
    return set(my_mapping.values())

f({3: 'yes', 4: 'no'})

You can even make your own duck types using Protocols and structural subtyping.

### Protocol

看下面例子

In [ ]:
def len(obj):
    return obj.__len__()

len()方法可以返回任何实现__len__魔法函数的对象的长度，那我们如何在len()里添加类型提示，尤其是参数obj的类型表示呢？

答案隐藏在学术术语structural subtyping[https://en.wikipedia.org/wiki/Structural_type_system]。structural subtyping的一种方法是根据它们是normal的还是structural的:
在normal系统中，类型之间的比较基于名称和声明。Python类型系统大多是名义上的，因为它们的子类型关系，可以用int来代替float。
在structural系统中，类型之间的比较基于结构。您可以定义一个结构类型“大小”，它包括定义的所有实例。__len__()，无论其标称类型如何.

目前正在通过PEP 544为Python带来一个成熟的结构类型系统，该系统旨在添加一个称为协议的概念。尽管大多数PEP 544已经在Mypy中实现了。

协议指定了一个或多个实现的方法。例如，所有类定义。__len__()完成typing.Sized协议。因此，我们可以将len()注释如下:

In [ ]:
from typing import Sized

def len(obj: Sized) -> int:
    return obj.__len__()

除此之外，在Typing中还包括以下模块 Container, Iterable, Awaitable, 还有 ContextManager.
你也可以声明自定的协议，通过导入typing_extensions模块中的Protocol协议对象，然后写一个继承该方法的子类，像下面这样:

In [ ]:
from typing_extensions import Protocol

class Sized(Protocol):
    def __len__(self) -> int: ...

def len(obj: Sized) -> int:
    return obj.__len__()

需要通过pip安装上面使用的第三方模块

pip install typing-extensions

在python中有一种公共模式，就是设置参数的默认值None，这样做通常是为了避免可变默认值的问题，或者让一个标记值标记特殊行为。

In [ ]:
from typing import Sequence, Optional

def player_order(
    names: Sequence[str], start: Optional[str] = None
) -> Sequence[str]:
    ...

等价于Union类型的 Union[None, str]，意思是这个参数的值类型为str，默认的话可以是 None

请注意，使用Optional或Union时，必须注意变量是否在后面有操作。比如上面的例子通过判断start是否为None。如果不判断None的情况，在做静态类型检查的时候会发生错误，因为Mypy告诉你还没有处理start为None的情况。

## Classes

In [ ]:
from typing import ClassVar

class MyClass:
    # You can optionally declare instance variables in the class body
    attr: int
    # This is an instance variable with a default value
    charge_percent: int = 100

    # The "__init__" method doesn't return anything, so it gets return
    # type "None" just like any other method that doesn't return anything
    def __init__(self) -> None:
        ...

    # For instance methods, omit type for "self"
    def my_method(self, num: int, str1: str) -> str:
        return num * str1

# User-defined classes are valid as types in annotations
x: MyClass = MyClass()

# You can use the ClassVar annotation to declare a class variable
class Car:
    seats: ClassVar[int] = 4
    passengers: ClassVar[List[str]]

In [ ]:
# You can also declare the type of an attribute in "__init__"
class Box:
    def __init__(self) -> None:
        self.items: List[str] = []

注意:__init__() 的返回值总是为None

### Forward references

In [ ]:
# Forward references are useful if you want to reference a class before
# it is defined
def f(foo: A) -> int:  # This will fail
    ...

class A:
    ...

# If you use the string literal 'A', it will pass as long as there is a
# class of that name later on in the file
def f(foo: 'A') -> int:  # Ok
    ...

通常，注释不会在运行时使用。这为推迟对注释的评估提供了动力。该提议不是将注释评估为Python表达式并存储其值，而是存储注释的字符串表示形式，并仅在需要时对其进行评估。

这种功能计划在Python 4.0中成为标准。但是，在Python 3.7及更高版本中，可以通过导入__future__属性的annotations来实现：

In [ ]:
from __future__ import annotations

class Deck:
    @classmethod
    def create(cls, shuffle: bool = False) -> Deck:
        ...
        

使用 __future__之后就可以使用Deck对象替换字符串"Deck"了。

### cls 和 self

一般来说，self 和 cls 不需要标注，但是 cls 有时也需要标注，比如在 classmethod 中，可能会返回待创建类的示例的情况。但是此时这个类还没有被创建，因此可以用 forward reference 的方法。m

In [ ]:
class A:
    def __init__(self, key: str, value: int) -> None:
        self.data = {key: value}
    @classmethod
    def from_str(cls, key: int, value: int) -> 'A':
        return cls(key, value)

### 继承关系中返回 self 或 cls

In [ ]:
from datetime import date

class Animal:
    def __init__(self, name: str, birthday: date) -> None:
        self.name = name
        self.birthday = birthday

    @classmethod
    def newborn(cls, name: str) -> "Animal":
        return cls(name, date.today())

    def twin(self, name: str) -> "Animal":
        cls = self.__class__
        return cls(name, self.birthday)

class Dog(Animal):
    def bark(self) -> None:
        print(f"{self.name} says woof!")

fido = Dog.newborn("Fido")
pluto = fido.twin("Pluto")
fido.bark()
pluto.bark()

运行上面的代码，Mypy会抛出下面的错误:

In [ ]:
$ mypy dogs.py
dogs.py:24: error: "Animal" has no attribute "bark"
dogs.py:25: error: "Animal" has no attribute "bark"

问题是，即使继承的Dog.newborn（）和Dog.twin（）方法将返回一个Dog，注释表明它们返回一个Animal。

在这种情况下，您需要更加小心以确保注释正确。返回类型应与self的类型或cls的实例类型匹配。这可以使用TypeVar来完成，这些变量会跟踪实际传递给self和cls的内容：

In [ ]:
# dogs.py

from datetime import date
from typing import Type, TypeVar

TAnimal = TypeVar("TAnimal", bound="Animal")

class Animal:
    def __init__(self, name: str, birthday: date) -> None:
        self.name = name
        self.birthday = birthday

    @classmethod
    def newborn(cls: Type[TAnimal], name: str) -> TAnimal:
        return cls(name, date.today())

    def twin(self: TAnimal, name: str) -> TAnimal:
        cls = self.__class__
        return cls(name, self.birthday)

class Dog(Animal):
    def bark(self) -> None:
        print(f"{self.name} says woof!")

fido = Dog.newborn("Fido")
pluto = fido.twin("Pluto")
fido.bark()
pluto.bark()

在这个例子中有几个需要注意的点：

类型变量TAnimal用于表示返回值可能是Animal的子类的实例。.

我们指定 Animal 是TAnimal的上限。指定绑定意味着TAnimal将是Animal子类之一。这可以正确限制所允许的类型。

typing.Type []是type()的类型。需要注意，是cls的类方法需要使用这种形式注解，而self就不用使用。

## Generic

### User-defined generic types[¶](#user-defined-generic-types "Permalink to this headline")

A user-defined class can be defined as a generic class.

In [ ]:
from typing import TypeVar, Generic
from logging import Logger

T = TypeVar('T')

class LoggedVar(Generic[T]):
    def __init__(self, value: T, name: str, logger: Logger) -> None:
        self.name = name
        self.logger = logger
        self.value = value

    def set(self, new: T) -> None:
        self.log('Set ' + repr(self.value))
        self.value = new

    def get(self) -> T:
        self.log('Get ' + repr(self.value))
        return self.value

    def log(self, message: str) -> None:
        self.logger.info('%s: %s', self.name, message)

### Alias for Generic

In [ ]:
from typing import TypeVar, Union, Iterable, Tuple

S = TypeVar('S')
# Response is a generic class
Response = Union[Iterable[S], int]

# Return type here is same as Union[Iterable[str], int]
def response(query: str) -> Response[str]:
    ...

T = TypeVar('T', int, float, complex)
Vec = Iterable[Tuple[T, T]]

def inproduct(v: Vec[T]) -> T: # Same as Iterable[tuple[T, T]]
    return sum(x*y for x, y in v)

注意，在指定 Generic 的时候，一定要加 []，否则不是 generic class 会报错。

In [ ]:
try:
    Response = Union[Iterable, int] # is not a generic
    s = Response[int]
except Exception as e:
    print(e)

## Miscellaneous

In [ ]:
import sys
import re
from typing import Match, AnyStr, IO

# "typing.Match" describes regex matches from the re module
x: Match[str] = re.match(r'[0-9]+', "15")

# Use IO[] for functions that should accept or return any
# object that comes from an open() call (IO[] does not
# distinguish between reading, writing or other modes)
def get_sys_IO(mode: str = 'w') -> IO[str]:
    if mode == 'w':
        return sys.stdout
    elif mode == 'r':
        return sys.stdin
    else:
        return sys.stdout

## Callables可调用类型

函数是Python中的一类对象。可以使用函数作为其他函数的参数。这意味着需要能够添加表示函数的类型提示。
函数以及lambdas、方法和类都由type的Callable对象表示。参数的类型和返回值通常也表示。例如，Callable[[A1, A2, A3],Rt]表示一个函数，它有三个参数，分别具有A1、A2和A3类型。函数的返回类型是Rt。

在下面这个例子, 函数 do_twice() 传入一个Callable类型的func参数，并指明传入的函数的参数类型为str，返回值类型为str。比如传入参数create_greeting.

In [ ]:
# do_twice.py

from typing import Callable

def do_twice(func: Callable[[str], str], argument: str) -> None:
    print(func(argument))

def create_greeting(name: str) -> str:
    return f"Hello {name}"

do_twice(create_greeting, "Jekyll")

## Decorators

Decorator functions can be expressed via generics. See Declaring decorators for more details.

In [ ]:
from typing import Any, Callable, TypeVar

F = TypeVar('F', bound=Callable[..., Any])

def bare_decorator(func: F) -> F:
    ...

def decorator_args(url: str) -> Callable[[F], F]:
    ...

## Type Aliases

我们可以使用起别名的方式把注解的类型赋值给一个新的变量，方便在后面使用，就像下面这样:

In [ ]:
from typing import List, Tuple

Card = Tuple[str, str]
Deck = List[Card]

def deal_hands(deck: Deck) -> Tuple[Deck, Deck, Deck, Deck]:
     """Deal the cards in the deck into four hands"""
     return (deck[0::4], deck[1::4], deck[2::4], deck[3::4])

## Type Variables[类型声明]

类型声明是一个特殊变量声明，可以采用任何类型，具体取决于具体情况。

In [ ]:
# choose.py

import random
from typing import Sequence, TypeVar

Choosable = TypeVar("Chooseable")

def choose(items: Sequence[Choosable]) -> Choosable:
  return random.choice(items)

names = ["Guido", "Jukka", "Ivan"]
reveal_type(names)

name = choose(names)
reveal_type(name)

In [ ]:
# choose.py

import random
from typing import Sequence, TypeVar

Choosable = TypeVar("Choosable", str, float)

def choose(items: Sequence[Choosable]) -> Choosable:
  return random.choice(items)

reveal_type(choose(["Guido", "Jukka", "Ivan"]))
reveal_type(choose([1, 2, 3]))
reveal_type(choose([True, 42, 3.14]))
reveal_type(choose(["Python", 3, 7]))

方法的类型提示与函数的类型提示非常相似。唯一的区别是self参数不需要注释，因为它是一个类的实例。Card类的类型很容易添加:

# References
- http://localhost:8888/lab/tree/learnPython/python-type-hints/Python%20type%20hint.ipynb
- https://my.oschina.net/u/4297117/blog/3199574
- [Type hints cheat sheet (Python 3) — Mypy 0.820+dev.bfc67b6129b3f82a7a6e92eddeedbcfb70440a31.dirty documentation](https://mypy.readthedocs.io/en/latest/cheat_sheet_py3.html#variables)
- [typing — Support for type hints — Python 3.9.2 documentation](https://docs.python.org/3/library/typing.html)